# PEI Group Interview Project

## Data Analyst Task

### The sales team has the following data from various sources:
* Customers.xls - [https://easyupload.io/t9m9my]
* Orders.csv - [https://easyupload.io/pngfna]
* Shippings.json - [https://easyupload.io/fm8t5t]


### Objectives: The team is trying to generate the reports for the below requirements:

* the total amount spent and the country for the Pending delivery status for each country.
* the total number of transactions, total quantity sold, and total amount spent for each customer, along with the product details.
* the maximum product purchased for each country.
* the most purchased product based on the age category less than 30 and above 30.
* the country that had minimum transactions and sales amount.


### Quality Checks: As a Data Analyst, you are required to

* Verify the accuracy, completeness, and reliability of source data. 
* Based on your findings, define and outline the requirements for anticipated datasets, detailing the necessary data components.
* Develop the data models to effectively organise and structure the information and provide a detailed mapping of existing data flows, focussing on the areas of concern.
* Prepare a story with technical specifications for one part of the data model for a data engineer.
* Communicate the findings and insights to stakeholders in a visually comprehensive manner.


In [1]:
# Additional packages required that are not part of the default docker image.

# Required to import xls files in pandas
!pip install xlrd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.1 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/pei-group/Order.csv
/kaggle/input/pei-group/Customer.xls
/kaggle/input/pei-group/Shipping.json


## Exploratory Data Analysis

### Importing Data
We will import data from multiple sources into our analysis environment using pandas.

In [3]:
customers = pd.read_excel('/kaggle/input/pei-group/Customer.xls', sheet_name=0)
orders = pd.read_csv('/kaggle/input/pei-group/Order.csv')
shipping = pd.read_json('/kaggle/input/pei-group/Shipping.json')

In [4]:
# Let's check if all dataframes have been created successfully
print(customers.head(), orders.head(), shipping.head(), sep='\n\n')

   Customer_ID    First     Last  Age Country
0            1   Joseph     Rice   43     USA
1            2     Gary    Moore   71     USA
2            3     John   Walker   44      UK
3            4     Eric   Carter   38      UK
4            5  William  Jackson   58     UAE

   Order_ID      Item  Amount  Customer_ID
0         1  Keyboard     400          139
1         2     Mouse     300          250
2         3   Monitor   12000          239
3         4  Keyboard     400          153
4         5  Mousepad     250          153

   Shipping_ID     Status  Customer_ID
0            1    Pending          173
1            2    Pending          155
2            3  Delivered          242
3            4    Pending          223
4            5  Delivered           72


#### Result
The schemas are matching with the data present in the files. Hence, the import was **successful**.

### Understanding Data

[Data Analysis Report.pdf](https://github.com/sarthak71/pei-interview-project/blob/main/Data%20Analysis%20Report.pdf) all information about the project such as:
* Data Dictionary (table schemas, available columns, column descriptions and assumed data types)
* Assumed constraints
* Business Process Diagram
* Data Flow Diagram

In [5]:
# Check the data size of each table.
print(f' Customer: {customers.shape} \n Order: {orders.shape} \n Shipping: {shipping.shape}', sep='\n\n')

 Customer: (250, 5) 
 Order: (250, 4) 
 Shipping: (250, 3)


In [6]:

print('Customers Table Info:')
customers.info()

print('\n\nOrders Table Info:')
orders.info()

print('\n\nShipping Table Info:')
shipping.info()

Customers Table Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  250 non-null    int64 
 1   First        250 non-null    object
 2   Last         250 non-null    object
 3   Age          250 non-null    int64 
 4   Country      250 non-null    object
dtypes: int64(2), object(3)
memory usage: 9.9+ KB


Orders Table Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     250 non-null    int64 
 1   Item         250 non-null    object
 2   Amount       250 non-null    int64 
 3   Customer_ID  250 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.9+ KB


Shipping Table Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3

#### Observations
* *Customer* table contains 250 rows and 5 columns. Hence, total data points = 1250.
* *Order* table contains 250 rows and 4 columns. Hence, total data points = 1000.
* *Shipping* table contains 250 rows and 3 columns. Hence, total data points = 750.
* *Customer*, *Order* & *Shipping* tables do not contain NULLs in any column.
* Age is not considered as continuous value because it can only contain integers.

### Data Cleaning

#### Tasks
1. Check for misspellings in fields: Country, Status, Item
    - See all the unique values in each field: Customer[Country], Shipping[Status], Order[Item].
2. Check missing data in all fields
    - See if any data cell is blank or NaN.
3. Check the tables should not contain any duplicates.
    - See if any primary key is repeating. Assume all other fields are allowed to be duplicated.
4. Check for Outliers based on Amounts & No. of Orders
    - See if any customer or a product stands out of the other, and decide how they should be treated.

In [7]:
# 1. Misspelling Checks
print(customers['Country'].value_counts().sort_values(), shipping['Status'].value_counts().sort_values(), orders['Item'].value_counts().sort_values(), sep='\n\n')

Country
UAE     49
UK     100
USA    101
Name: count, dtype: int64

Status
Delivered    100
Pending      150
Name: count, dtype: int64

Item
Mouse       25
Monitor     25
Harddisk    25
Webcam      25
DDR RAM     25
Headset     25
Keyboard    50
Mousepad    50
Name: count, dtype: int64


#### Conclusion
* There are no misspelled data in *Country*, *Status* and *Item* columns.
* *Country*, *Status* and *Item* columns have consistent datatypes.
* *Country*, *Status* and *Item* columns follow Atomicity, i.e. one data point contains only one value

In [8]:
# 2. Check if there are missing values.

_var = (customers == 0 | customers.isna() | customers.map(lambda x: str(x).strip() == '')).sum()
print(f'No. of Missing data points in Customers table:\n {_var}\n\n')
_var = (orders == 0 | orders.isna() | orders.map(lambda x: str(x).strip() == '')).sum()
print(f'No. of Missing data points in Orders table:\n {_var}\n\n')
_var = (shipping == 0 | shipping.isna() | shipping.map(lambda x: str(x).strip() == '')).sum()
print(f'No. of Missing data points in Shipping table:\n {_var}\n\n')

No. of Missing data points in Customers table:
 Customer_ID    0
First          0
Last           0
Age            0
Country        0
dtype: int64


No. of Missing data points in Orders table:
 Order_ID       0
Item           0
Amount         0
Customer_ID    0
dtype: int64


No. of Missing data points in Shipping table:
 Shipping_ID    0
Status         0
Customer_ID    0
dtype: int64




#### Conclusion
* There are 250 rows in each table, and all columns have 250 non-null values
* There are no NULLs, zeroes, or \<Blanks\> in all three tables.

In [9]:
# 3. Check duplicates in Primary Key
print('# Duplicates in Customers table: ', end='')
print(customers.duplicated(subset=['Customer_ID']).sum())

print('# Duplicates in Orders table: ', end='')
print(orders.duplicated(subset=['Order_ID']).sum())

print('# Duplicates in Shipping table: ', end='')
print(shipping.duplicated(subset=['Shipping_ID']).sum())

# Duplicates in Customers table: 0
# Duplicates in Orders table: 0
# Duplicates in Shipping table: 0


#### Conclusion

* Assumed primary keys *Customer_ID*, *Order_ID* and *Shipping_ID* follow all data consistency constraints.
* There is only one item purchased per order.

In [10]:
# 4. Checking for Outliers

print('For Customers Table:')
tstats = customers.describe().transpose()
tstats['iqr'] = tstats['75%'] - tstats['25%']
tstats['valid_range_from'] = tstats['25%'] - (tstats['iqr']*1.5)
tstats['valid_range_to'] = tstats['75%'] + (tstats['iqr']*1.5)
print(tstats.transpose(), end='\n\n')

print('For Orders Table')
tstats = orders.describe().transpose()
tstats['iqr'] = tstats['75%'] - tstats['25%']
tstats['valid_range_from'] = tstats['25%'] - (tstats['iqr']*1.5)
tstats['valid_range_to'] = tstats['75%'] + (tstats['iqr']*1.5)
print(tstats.transpose(), end='\n\n')

print('For Shipping Table')
tstats = shipping.describe().transpose()
tstats['iqr'] = tstats['75%'] - tstats['25%']
tstats['valid_range_from'] = tstats['25%'] - (tstats['iqr']*1.5)
tstats['valid_range_to'] = tstats['75%'] + (tstats['iqr']*1.5)
print(tstats.transpose(), end='\n\n')

For Customers Table:
                  Customer_ID         Age
count              250.000000  250.000000
mean               125.500000   47.576000
std                 72.312977   18.978011
min                  1.000000   18.000000
25%                 63.250000   29.000000
50%                125.500000   47.000000
75%                187.750000   63.000000
max                250.000000   80.000000
iqr                124.500000   34.000000
valid_range_from  -123.500000  -22.000000
valid_range_to     374.500000  114.000000

For Orders Table
                    Order_ID       Amount  Customer_ID
count             250.000000    250.00000   250.000000
mean              125.500000   2130.00000   130.404000
std                72.312977   3575.43493    69.192711
min                 1.000000    200.00000     4.000000
25%                63.250000    300.00000    71.500000
50%               125.500000    400.00000   125.500000
75%               187.750000   1500.00000   190.750000
max              

In [11]:
# From orders table, the max of Amount value is outside the valid data range (0, 3300), hence this requires further analysis.
# Exploring Amount outliers further:

# Let's analyse the items that are valued above 3300.
orders[orders['Amount'] > 3300].groupby('Item').agg({'Order_ID': 'count', 'Amount': ['mean', 'sum', 'count']})


Order_ID   Amount              
            count     mean     sum count
Item                                    
Harddisk       25   5000.0  125000    25
Monitor        25  12000.0  300000    25

#### Conclusion
* The outliers have been identified based on the statistical method which considers outliers as values outside the range from `[Quartile_1 - (IQR * 1.5)] to [Quartile_3 + (IQR * 1.5)]`. This method especially works with data that does not follow normal curve.
* From customers table, we don't see any customer outside the *Age* range (0, 114), hence there are no outliers. Since age cannot be negative, we can assume the range starts from 0.
* From orders table, the max of *Amount* value is **outside** the valid data range (0, 3300). On further analysis, we found that there are significant number of orders with the amount above 3300. Hence, no action is required.
* shipping table does not contain numeric data, except the identifiers.

### Data Validation

* Do all entries in a column have same datatype?
* Do all entries lie in the same/expected range?
* Do all entries have the same format?


In [12]:
print('Data Summary of Customer table:', customers.describe(include='all'), sep='\n\n', end='\n\n')
print('Data Summary of Order table:', orders.describe(include='all'), sep='\n\n', end='\n\n')
print('Data Summary of Shipping table:', shipping.describe(include='all'), sep='\n\n', end='\n\n')

Data Summary of Customer table:

        Customer_ID   First      Last         Age Country
count    250.000000     250       250  250.000000     250
unique          NaN     171       189         NaN       3
top             NaN  Joseph  Williams         NaN     USA
freq            NaN       5         6         NaN     101
mean     125.500000     NaN       NaN   47.576000     NaN
std       72.312977     NaN       NaN   18.978011     NaN
min        1.000000     NaN       NaN   18.000000     NaN
25%       63.250000     NaN       NaN   29.000000     NaN
50%      125.500000     NaN       NaN   47.000000     NaN
75%      187.750000     NaN       NaN   63.000000     NaN
max      250.000000     NaN       NaN   80.000000     NaN

Data Summary of Order table:

          Order_ID      Item       Amount  Customer_ID
count   250.000000       250    250.00000   250.000000
unique         NaN         8          NaN          NaN
top            NaN  Keyboard          NaN          NaN
freq           NaN  

In [13]:
# First Name and Last Names should not contain extra spaces and special characters.
customers.loc[~customers['First'].str.isalpha()]

,Customer_ID,First,Last,Age,Country
5,6,N!cole,Jones,33,USA
13,14,N!cole,Lara,77,UK
108,109,R0bert,Moore,40,UK
117,118,R0bert,Shepherd,28,UK
161,162,N!cole,Bennett,51,USA
170,171,L@rry,Cole,50,USA
197,198,R0bert,Bryan,49,UK
210,211,Al1cia,Thompson,38,USA
213,214,N!cole,Mcintyre,18,UK
235,236,Al1cia,Jensen,19,USA


In [14]:
customers.loc[~customers['Last'].str.isalpha()]

,Customer_ID,First,Last,Age,Country
112,113,Derrick,R0berts,72,UK
241,242,Mark,R0berts,61,USA


#### Conclusion

* As established, we can see that `Customer_ID`, `Order_ID` and `Shipping_ID` primary key columns have no missing values, no duplicates and min and max values are 1 and 250 respectively. Hence, these columns contain all integers between 1 and 250, both inclusive.
* `Customer_ID` foreign key has values from 4 to 250 in Orders table and 1 to 248 in Shipping table. Hence, there is one primary key for each foreign key here.
* `Age` column ranges from 18 to 80, which indicates the orders were placed by customers with appropriate age.
* `Status` column has only 2 unique values as expected.
* `Item` column has 8 unique values as expected.

Hence,
* All data entries follow the column's data type constraints.
* All data falls under the expected range of values.
* All values follow the same format across the field/column.


### Data Anomalies Detection

In [15]:
# 1. Have all orders have been shipped?
# All customers in Order table must have an entry in Shipping table
# This is not an ideal scenario, usually shipment is related to order and order is further related to customer.

n_not_started_orders = orders.loc[~orders['Customer_ID'].isin(shipping['Customer_ID']), ['Order_ID']].shape[0]
print(f'There are {n_not_started_orders} orders for which shipment has not been started.')

There are 94 orders for which shipment has not been started.


In [16]:
# 2. Number of customers who placed multiple orders should be same as customers who have multiple orders placed.
# Are there multiple shipments to a customer?
cust = shipping.groupby('Customer_ID')['Shipping_ID'].count()
print(f'There are {cust[cust > 1].size} customers who have multiple shipments')

# Has any customer placed multiple orders?
cust = orders.groupby('Customer_ID')['Order_ID'].count()
print(f'There are {cust[cust > 1].size} customers who have placed multiple orders.')

There are 67 customers who have multiple shipments
There are 60 customers who have placed multiple orders.


In [17]:
# 3. Is there any shipment for which order doesn’t exist?
n_shipment_wo_order = shipping.loc[~shipping['Customer_ID'].isin(orders['Customer_ID']), ['Shipping_ID']].shape[0]
print(f'There are {n_shipment_wo_order} shipments for which order could not be found.')

There are 98 shipments for which order could not be found.


In [18]:
# 4. Is there any item sold at 2 or more different amount?
orders.groupby(['Item', 'Amount']).count().sort_values(by='Item').reset_index()

,Item,Amount,Order_ID,Customer_ID
0,DDR RAM,1500,25,25
1,Harddisk,5000,25,25
2,Headset,900,25,25
3,Keyboard,400,50,50
4,Monitor,12000,25,25
5,Mouse,300,25,25
6,Mousepad,200,25,25
7,Mousepad,250,25,25
8,Webcam,350,25,25


## Data Preparation & Visualization
All data preparation and visualization has been done in PowerBI.
Here are the links:
* [Dashboard (PDF)](https://github.com/sarthak71/pei-interview-project/blob/main/Order_and_Shipping_Dashboard.pdf)
* [Dashboard (.pbix)](https://github.com/sarthak71/pei-interview-project/blob/main/Order_and_Shipping_Dashboard.pbix)